In [1]:
!pip install nixtlats

In [59]:
df = pd.read_csv("data-8013-trends.csv")
df.head()

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,BILLING_DATE,SITE,BRAND,FORMAT_DESC,MH_SEGMENT,MH_FAMILY,MH_CLASS,MH_BRICK,ARTICLE,TOTAL_BILLING_QTY,TOTAL_GROSS_MARGIN,TOTAL_MARKDOWN,TOTAL_NET_SALES,TOTAL_GROSS_SALES,TOTAL_DISCOUNT
0,712337,712337,712337,712337,712337,2022-09-30,8013,04017,TRENDS,WOMENS WEAR,WESTERN WEAR,WINTER WEAR,SWEATSHIRTS,441146771010,1,78.66,0.0,334.28,499.0,164.72
1,669408,669408,669408,669408,669408,2022-09-30,8013,D525,TRENDS,WOMENS WEAR,WESTERN WEAR,CASUAL WEAR,T SHIRTS,441137882004,1,71.91,0.0,240.26,399.0,158.74
2,236195,236195,236195,236195,236195,2022-09-30,8013,09888,TRENDS,MENS CASUAL,ACTIVE WEAR,TOPS,T SHIRTS,441134761003,1,88.72,0.0,322.58,599.0,276.42
3,298014,298014,298014,298014,298014,2022-09-30,8013,04063,TRENDS,MENS CASUAL,ACTIVE WEAR,TOPS,T SHIRTS,441145712004,1,408.46,0.0,799.00,799.0,0.00
4,776725,776725,776725,776725,776725,2022-09-30,8013,04042,TRENDS,MENS WEAR,SMART CASUALS,BOTTOMS,TROUSERS,441138083006,1,239.82,0.0,790.98,999.0,208.02


In [54]:
df['BILLING_DATE'] = pd.to_datetime(df['BILLING_DATE'])

df.to_csv( "data-8013-trends.csv")

In [57]:
import pandas as pd
from io import StringIO

# Load your data (assuming it's in CSV format)

# Sort by BILLING_DATE
df_sorted = df.sort_values(by='BILLING_DATE')
df_sorted.to_csv( "data-8013-trends.csv")


In [27]:
# The error occurs because 'shape' is not a standalone function; it is an attribute of DataFrame objects.
# To get the shape of df_sorted, use:
df_sorted.shape

(823225, 18)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from nixtlats import TimeGPT
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Initialize TimeGPT client
timegpt = TimeGPT(token='nixak-AI31YupjpWhin07kPGKOhvW5zj8IliWOlTSWLEInpOuKHNWNhtSETsXIFgyEiYT58g3Hk0hvMFhnJpdS')  # Replace with your actual token

def complete_timegpt_pipeline_with_exogenous(df_sorted, top_n_products=8):
    """
    Complete end-to-end TimeGPT forecasting pipeline with exogenous features
    """
    print("🚀 STARTING ENHANCED TIMEGPT PIPELINE WITH EXOGENOUS FEATURES")
    print("=" * 70)
    
    # Step 1: Enhanced Data Cleaning and Preparation with Exogenous Features
    print("\n📋 STEP 1: DATA CLEANING WITH EXOGENOUS FEATURES")
    cleaned_data, predictable_features = clean_and_prepare_data_with_exogenous(df_sorted, top_n_products)
    
    # Step 2: Fix Frequency Issues
    print("\n🔧 STEP 2: FIXING FREQUENCY ISSUES")
    frequency_fixed_data = fix_frequency_issues_complete(cleaned_data)
    
    # Step 3: Train-Test Split
    print("\n✂️ STEP 3: TRAIN-TEST SPLIT")
    train_data, test_data = create_train_test_split(frequency_fixed_data)
    
    # Step 4: TimeGPT Forecasting with Exogenous Features
    print("\n🤖 STEP 4: TIMEGPT FORECASTING WITH EXOGENOUS FEATURES")
    forecasts = run_timegpt_forecasting_fixed(train_data, predictable_features, test_periods=7)
    
    # Step 5: Evaluation
    print("\n📊 STEP 5: MODEL EVALUATION")
    evaluation_results = evaluate_forecasts(forecasts, test_data)
    
    # Step 6: Visualization
    print("\n📈 STEP 6: CREATING VISUALIZATIONS")
    create_comprehensive_visualizations(frequency_fixed_data, forecasts, test_data, evaluation_results)
    
    # Step 7: Future Forecasting
    print("\n🔮 STEP 7: FUTURE FORECASTING")
    future_forecasts = generate_future_forecasts(frequency_fixed_data, horizon=14)  # Reduced horizon
    
    return {
        'cleaned_data': frequency_fixed_data,
        'train_data': train_data,
        'test_data': test_data,
        'forecasts': forecasts,
        'evaluation': evaluation_results,
        'future_forecasts': future_forecasts,
        'exogenous_features': predictable_features
    }

def clean_and_prepare_data_with_exogenous(df_sorted, top_n_products=8):
    """
    Enhanced data cleaning with exogenous feature integration
    """
    print("   • Creating exogenous features before data aggregation...")
    
    # ✅ Step 1: Create exogenous features on raw data FIRST
    df_enhanced, predictable_features = prepare_exogenous_data_for_timegpt(df_sorted)
    
    print(f"   • Created {len(predictable_features)} predictable exogenous features")
    
    # Handle negative sales values
    print(f"   • Found {(df_enhanced['TOTAL_NET_SALES'] < 0).sum()} negative sales records")
    df_enhanced.loc[df_enhanced['TOTAL_NET_SALES'] < 0, 'TOTAL_NET_SALES'] = 0
    
    # Create PRODUCT_KEY if not exists
    if 'PRODUCT_KEY' not in df_enhanced.columns:
        df_enhanced['PRODUCT_KEY'] = (
            df_enhanced['BRAND'].astype(str) + ' | ' + 
            df_enhanced['MH_SEGMENT'].astype(str) + ' | ' + 
            df_enhanced['MH_FAMILY'].astype(str) + ' | ' + 
            df_enhanced['MH_CLASS'].astype(str) + ' | ' + 
            df_enhanced['MH_BRICK'].astype(str)
        )
    
    # Product filtering (same as before)
    product_analysis = df_enhanced.groupby('PRODUCT_KEY').agg({
        'TOTAL_NET_SALES': ['sum', 'mean', 'count', 'std'],
        'BILLING_DATE': ['min', 'max', 'nunique'],
        'TOTAL_BILLING_QTY': 'sum',
        'TOTAL_GROSS_MARGIN': 'sum'
    }).round(2)
    
    product_analysis.columns = ['total_sales', 'avg_sales', 'transaction_count','sales_std', 'first_date', 'last_date', 'unique_days','total_qty', 'total_margin']
    
    # Calculate quality metrics
    product_analysis['data_span_days'] = (product_analysis['last_date'] - product_analysis['first_date']).dt.days + 1
    product_analysis['data_density'] = product_analysis['unique_days'] / product_analysis['data_span_days']
    product_analysis['cv'] = product_analysis['sales_std'] / product_analysis['avg_sales']
    
    # Product filtering
    viable_products = product_analysis[
        (product_analysis['unique_days'] >= 60) &
        (product_analysis['data_span_days'] >= 90) &
        (product_analysis['data_density'] >= 0.3) &
        (product_analysis['total_sales'] >= 10000) &
        (product_analysis['transaction_count'] >= 30) &
        (product_analysis['cv'] < 3)
    ].sort_values('total_sales', ascending=False)
    
    print(f"   📊 Product filtering with exogenous features:")
    print(f"      • Viable for forecasting: {len(viable_products)}")
    print(f"      • Selected top: {min(top_n_products, len(viable_products))}")
    
    if len(viable_products) == 0:
        viable_products = product_analysis[
            (product_analysis['unique_days'] >= 30) &
            (product_analysis['total_sales'] >= 5000)
        ].sort_values('total_sales', ascending=False)
    
    selected_products = viable_products.head(min(top_n_products, len(viable_products))).index.tolist()
    filtered_df = df_enhanced[df_enhanced['PRODUCT_KEY'].isin(selected_products)].copy()
    
    print("   • Aggregating with enhanced exogenous variables...")
    
    # ✅ Step 2: Aggregate including exogenous features
    agg_dict = {
        'TOTAL_NET_SALES': 'sum',
        'TOTAL_BILLING_QTY': 'sum',
        'TOTAL_GROSS_MARGIN': 'sum',
        'TOTAL_GROSS_SALES': 'sum',
        'TOTAL_DISCOUNT': 'sum'
    }
    
    # Add predictable exogenous features to aggregation (using 'first' since they're constant per day)
    for feature in predictable_features:
        if feature in filtered_df.columns:
            agg_dict[feature] = 'first'
    
    daily_sales = filtered_df.groupby(['BILLING_DATE', 'PRODUCT_KEY']).agg(agg_dict).reset_index()
    
    # Rename columns for TimeGPT
    column_mapping = {
        'BILLING_DATE': 'ds',
        'TOTAL_NET_SALES': 'y',
        'PRODUCT_KEY': 'unique_id',
        'TOTAL_BILLING_QTY': 'qty',
        'TOTAL_GROSS_MARGIN': 'margin',
        'TOTAL_GROSS_SALES': 'gross_sales',
        'TOTAL_DISCOUNT': 'discount'
    }
    
    timegpt_data = daily_sales.rename(columns=column_mapping)
    
    # Data type optimization
    timegpt_data['ds'] = pd.to_datetime(timegpt_data['ds'])
    for col in ['y', 'qty', 'margin', 'gross_sales', 'discount']:
        if col in timegpt_data.columns:
            timegpt_data[col] = pd.to_numeric(timegpt_data[col], errors='coerce')
    
    # Remove invalid records
    timegpt_data = timegpt_data.dropna(subset=['y'])
    
    print(f"   ✅ Enhanced data with exogenous features: {len(timegpt_data):,} rows, {timegpt_data['unique_id'].nunique()} products")
    print(f"   📊 Exogenous features included: {len([f for f in predictable_features if f in timegpt_data.columns])}")
    
    return timegpt_data, predictable_features


def create_calendar_features(df):
    """Create comprehensive calendar features for Indian retail"""
    df = df.copy()
    df['BILLING_DATE'] = pd.to_datetime(df['BILLING_DATE'])
    
    # Basic temporal features
    df['year'] = df['BILLING_DATE'].dt.year
    df['month'] = df['BILLING_DATE'].dt.month
    df['day_of_week'] = df['BILLING_DATE'].dt.dayofweek
    df['day_of_year'] = df['BILLING_DATE'].dt.dayofyear
    df['week_of_year'] = df['BILLING_DATE'].dt.isocalendar().week
    df['quarter'] = df['BILLING_DATE'].dt.quarter
    
    # Weekend indicator
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    
    # Month-end/start effects
    df['is_month_start'] = df['BILLING_DATE'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['BILLING_DATE'].dt.is_month_end.astype(int)
    df['is_quarter_start'] = df['BILLING_DATE'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['BILLING_DATE'].dt.is_quarter_end.astype(int)
    
    return df


def create_indian_holiday_features(df):
    """Add Indian holiday indicators based on 2024 calendar"""
    df = df.copy()
    
    # Major Indian holidays for 2024 (extend for other years)
    major_holidays = {
        '2024-01-26': 'republic_day',
        '2024-03-25': 'holi', 
        '2024-04-11': 'eid_ul_fitr',
        '2024-04-17': 'ram_navami',
        '2024-08-15': 'independence_day',
        '2024-08-26': 'janmashtami',
        '2024-10-12': 'dussehra',
        '2024-10-31': 'diwali',
        '2024-12-25': 'christmas'
    }
    
    # Create holiday indicators
    for date_str, holiday_name in major_holidays.items():
        df[f'is_{holiday_name}'] = (df['BILLING_DATE'].dt.strftime('%Y-%m-%d') == date_str).astype(int)
    
    # ✅ FIXED: Holiday proximity features (corrected variable names)
    diwali_date = pd.to_datetime('2024-10-31')
    for days in [1, 2, 3]:
        df[f'days_before_diwali_{days}'] = (df['BILLING_DATE'] == diwali_date - pd.Timedelta(days=days)).astype(int)
        df[f'days_after_diwali_{days}'] = (df['BILLING_DATE'] == diwali_date + pd.Timedelta(days=days)).astype(int)
    
    return df




def create_business_features(df):
    """Create features from existing business data"""
    df = df.copy()
    
    # Categorical encoding for key business dimensions
    categorical_cols = ['SITE', 'BRAND', 'FORMAT_DESC', 'MH_SEGMENT', 'MH_FAMILY']
    
    for col in categorical_cols:
        # Create frequency encoding
        freq_map = df[col].value_counts(normalize=True).to_dict()
        df[f'{col}_frequency'] = df[col].map(freq_map)
        
        # One-hot encode top categories only (to avoid high dimensionality)
        top_categories = df[col].value_counts().head(5).index
        for category in top_categories:
            df[f'{col}_{category}'] = (df[col] == category).astype(int)
    
    # Price-related features
    df['avg_selling_price'] = df['TOTAL_NET_SALES'] / df['TOTAL_BILLING_QTY'].replace(0, 1)
    df['discount_rate'] = df['TOTAL_DISCOUNT'] / df['TOTAL_GROSS_SALES'].replace(0, 1)
    df['margin_rate'] = df['TOTAL_GROSS_MARGIN'] / df['TOTAL_NET_SALES'].replace(0, 1)
    
    return df


def prepare_exogenous_data_for_timegpt(df, forecast_horizon=14):
    """Prepare exogenous data in TimeGPT format"""
    
    # Step 1: Create all features
    df_enhanced = create_calendar_features(df)
    df_enhanced = create_indian_holiday_features(df_enhanced)
    df_enhanced = create_business_features(df_enhanced)
    
    # Step 2: Separate predictable vs unpredictable features
    predictable_features = [
        'year', 'month', 'day_of_week', 'is_weekend', 'is_month_start', 
        'is_month_end', 'is_quarter_start', 'is_quarter_end',
        'is_republic_day', 'is_holi', 'is_diwali'  # Add all holiday features
    ]
        
    return df_enhanced, predictable_features



def fix_frequency_issues_complete(data):
    """
    Robust frequency fixing optimized for your irregular data patterns
    """
    print("   • Applying robust frequency correction for irregular data...")
    
    cleaned_data = []
    successful_products = []
    
    for product_id, group in data.groupby('unique_id'):
        # Sort and remove duplicates
        group = group.sort_values('ds').drop_duplicates(subset=['ds'], keep='first')
        
        print(f"      Processing: {product_id[:40]}... ({len(group)} records)")
        
        # Skip products with insufficient data (stricter threshold)
        if len(group) < 50:  # Increased from 30 to 50
            print(f"        ❌ Insufficient data: {len(group)} < 50 days")
            continue
        
        # Analyze existing frequency pattern
        date_diffs = group['ds'].diff().dropna()
        
        # Create complete date range
        date_range = pd.date_range(
            start=group['ds'].min(),
            end=group['ds'].max(),
            freq='D'
        )
        
        expected_points = len(date_range)
        missing_points = expected_points - len(group)
        missing_ratio = missing_points / expected_points
        
        print(f"        📊 Missing ratio: {missing_ratio:.2%}")
        
        # Skip products with too much missing data
        if missing_ratio > 0.7:  # More than 70% missing
            print(f"        ❌ Too sparse: {missing_ratio:.1%} missing data")
            continue
        
        # Create complete time series
        complete_ts = pd.DataFrame({
            'ds': date_range,
            'unique_id': product_id
        })
        
        # Merge with actual data
        merged = pd.merge(complete_ts, group, on=['ds', 'unique_id'], how='left')
        
        # Enhanced interpolation strategy based on missing ratio
        numeric_cols = ['y', 'qty', 'margin', 'gross_sales', 'discount']
        
        for col in numeric_cols:
            if col in merged.columns:
                if missing_ratio < 0.2:  # Less than 20% missing - linear interpolation
                    merged[col] = merged[col].interpolate(method='linear')
                elif missing_ratio < 0.4:  # 20-40% missing - limit interpolation span
                    merged[col] = merged[col].interpolate(method='linear', limit=5)
                else:  # 40-70% missing - very conservative interpolation
                    merged[col] = merged[col].interpolate(method='linear', limit=2)
                
                # Forward/backward fill for remaining gaps, then zero
                merged[col] = merged[col].fillna(method='ffill', limit=3)
                merged[col] = merged[col].fillna(method='bfill', limit=3)
                merged[col] = merged[col].fillna(0)
                merged[col] = merged[col].clip(lower=0)  # Ensure non-negative
        
        # Verify frequency can be inferred
        freq_check = pd.infer_freq(merged['ds'])
        if freq_check == 'D':
            cleaned_data.append(merged)
            successful_products.append(product_id)
            print(f"        ✅ Success: {missing_points} gaps filled")
        else:
            print(f"        ❌ Frequency check failed")
    
    if cleaned_data:
        final_data = pd.concat(cleaned_data, ignore_index=True)
        print(f"   ✅ Robust processing complete: {len(successful_products)} products")
        print(f"   📊 Final dataset: {len(final_data):,} rows")
        return final_data
    else:
        raise ValueError("No products survived robust frequency correction")

def create_train_test_split(data, test_days=21):
    """
    Create train-test split
    """
    print(f"   • Splitting data with {test_days} test days...")
    
    # Split data by taking last N days for testing
    test_data = data.groupby("unique_id").tail(test_days)
    train_data = (data.groupby("unique_id").apply(lambda group: group.iloc[:-test_days]).reset_index(drop=True))
    
    print(f"   ✅ Train: {len(train_data)} rows, Test: {len(test_data)} rows")
    
    return train_data, test_data

def run_timegpt_forecasting_fixed(train_data, predictable_features, test_periods=7):
    """
    CORRECTED TimeGPT forecasting with proper exogenous variable format
    """
    print(f"   🎯 Running CORRECTED TimeGPT forecast for {test_periods} periods...")
    
    try:
        # ✅ CORRECT APPROACH: Separate main dataframe and exogenous variables
        available_exog = [f for f in predictable_features if f in train_data.columns]
        
        if available_exog:
            print(f"   • Separating exogenous variables: {available_exog}")
            
            # Main dataframe with only target variable
            main_data = train_data[['unique_id', 'ds', 'y']].copy()
            
            # Separate exogenous dataframe
            X_df = train_data[['unique_id', 'ds'] + available_exog].copy()
            
            print(f"   • Main data shape: {main_data.shape}")
            print(f"   • Exogenous data shape: {X_df.shape}")
            print(f"   • Exogenous columns: {available_exog}")
            
            # Generate forecasts with separate dataframes
            forecasts = timegpt.forecast(
                df=main_data,          # ✅ Main dataframe with target only
                X_df=X_df,            # ✅ Separate exogenous dataframe
                h=test_periods,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id',
                # Fine-tuning parameters
                finetune_steps=30,
                finetune_loss='mae',
                clean_ex_first=True
            )
            
        else:
            print("   • No exogenous variables available, using target only")
            main_data = train_data[['unique_id', 'ds', 'y']].copy()
            
            forecasts = timegpt.forecast(
                df=main_data,
                h=test_periods,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id',
                finetune_steps=30,
                finetune_loss='mae'
            )
        
        print(f"   ✅ CORRECTED forecasts generated: {len(forecasts):,} rows")
        return forecasts
        
    except Exception as e:
        print(f"   ❌ Corrected forecasting failed: {e}")
        print("   🔄 Trying basic approach without exogenous variables...")
        
        # Fallback: Use only target variables
        try:
            basic_data = train_data[['unique_id', 'ds', 'y']].copy()
            
            forecasts = timegpt.forecast(
                df=basic_data,
                h=test_periods,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id',
                finetune_steps=20,
                finetune_loss='mae'
            )
            
            print(f"   ✅ Basic forecasts generated: {len(forecasts):,} rows")
            return forecasts
            
        except Exception as e2:
            print(f"   ❌ All approaches failed: {e2}")
            
            # Final fallback - minimal configuration
            minimal_data = train_data[['unique_id', 'ds', 'y']].head(1000)  # Reduce data size
            
            forecasts = timegpt.forecast(
                df=minimal_data,
                h=test_periods,
                freq='D',
                time_col='ds',
                target_col='y',
                id_col='unique_id'
            )
            
            print(f"   ✅ Minimal fallback forecasts: {len(forecasts):,} rows")
            return forecasts


def generate_future_forecasts(data, horizon=14):
    """
    Generate future forecasts with proper exogenous variable handling
    """
    print(f"   • Generating {horizon}-day future forecasts...")
    
    try:
        # Check if exogenous variables exist
        exog_cols = ['year', 'month', 'day_of_week', 'is_weekend', 'is_month_start', 
                    'is_month_end', 'is_quarter_start', 'is_quarter_end',
                    'is_republic_day', 'is_holi', 'is_diwali']
        
        available_exog = [col for col in exog_cols if col in data.columns]
        
        if available_exog:
            # Create future exogenous data
            future_X_df = create_future_exogenous_data(data, available_exog, horizon)
            
            # Main data for forecasting
            main_data = data[['unique_id', 'ds', 'y']].copy()
            
            future_forecasts = timegpt.forecast(
                df=main_data,
                X_df=future_X_df,  # Future exogenous data
                h=horizon,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id'
            )
        else:
            # No exogenous variables
            future_forecasts = timegpt.forecast(
                df=data[['unique_id', 'ds', 'y']],
                h=horizon,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id'
            )
        
        # Create summary
        forecast_summary = future_forecasts.groupby('unique_id').agg({
            'TimeGPT': ['mean', 'sum']
        }).round(2)
        
        forecast_summary.columns = ['avg_daily', 'total_forecast']
        forecast_summary = forecast_summary.sort_values('total_forecast', ascending=False)
        
        print(f"   ✅ Future forecasts generated")
        print(f"   📊 Top 5 products by projected sales:")
        print(forecast_summary.head())
        
        return future_forecasts
        
    except Exception as e:
        print(f"   ❌ Future forecast error: {e}")
        return None


def create_future_exogenous_data(train_data, exog_features, horizon):
    """
    Create future exogenous variable values for forecasting
    """
    future_data = []
    
    for unique_id in train_data['unique_id'].unique():
        # Get last date for this product
        product_data = train_data[train_data['unique_id'] == unique_id]
        last_date = product_data['ds'].max()
        
        # Generate future dates
        future_dates = pd.date_range(
            start=last_date + pd.Timedelta(days=1),
            periods=horizon,
            freq='D'
        )
        
        for date in future_dates:
            row = {'unique_id': unique_id, 'ds': date}
            
            # Add predictable calendar features
            if 'year' in exog_features:
                row['year'] = date.year
            if 'month' in exog_features:
                row['month'] = date.month
            if 'day_of_week' in exog_features:
                row['day_of_week'] = date.dayofweek
            if 'is_weekend' in exog_features:
                row['is_weekend'] = int(date.dayofweek >= 5)
            if 'is_month_start' in exog_features:
                row['is_month_start'] = int(date.is_month_start)
            if 'is_month_end' in exog_features:
                row['is_month_end'] = int(date.is_month_end)
            if 'is_quarter_start' in exog_features:
                row['is_quarter_start'] = int(date.is_quarter_start)
            if 'is_quarter_end' in exog_features:
                row['is_quarter_end'] = int(date.is_quarter_end)
            
            # Add holiday indicators (extend for future years)
            if 'is_republic_day' in exog_features:
                row['is_republic_day'] = int(date.strftime('%m-%d') == '01-26')
            if 'is_holi' in exog_features:
                row['is_holi'] = 0  # Add logic for variable date holidays
            if 'is_diwali' in exog_features:
                row['is_diwali'] = 0  # Add logic for variable date holidays
            
            future_data.append(row)
    
    return pd.DataFrame(future_data)

# def create_future_exogenous_data(train_data, exog_features, horizon):
#     """
#     Create future exogenous variable values for forecasting
#     """
#     future_data = []
    
#     for unique_id in train_data['unique_id'].unique():
#         # Get last date for this product
#         product_data = train_data[train_data['unique_id'] == unique_id]
#         last_date = product_data['ds'].max()
        
#         # Generate future dates
#         future_dates = pd.date_range(
#             start=last_date + pd.Timedelta(days=1),
#             periods=horizon,
#             freq='D'
#         )
        
#         for date in future_dates:
#             row = {'unique_id': unique_id, 'ds': date}
            
#             # Add predictable calendar features
#             row['year'] = date.year
#             row['month'] = date.month
#             row['day_of_week'] = date.dayofweek
#             row['is_weekend'] = int(date.dayofweek >= 5)
#             row['is_month_start'] = int(date.is_month_start)
#             row['is_month_end'] = int(date.is_month_end)
#             row['is_quarter_start'] = int(date.is_quarter_start)
#             row['is_quarter_end'] = int(date.is_quarter_end)
            
#             # Add holiday indicators (extend for future years)
#             row['is_republic_day'] = int(date.strftime('%m-%d') == '01-26')
#             row['is_holi'] = 0  # Add logic for variable date holidays
#             row['is_diwali'] = 0  # Add logic for variable date holidays
            
#             future_data.append(row)
    
#     return pd.DataFrame(future_data)


def evaluate_forecasts(forecasts, test_data):
    """
    Evaluate forecast performance with improved MAPE handling
    """
    print("   • Calculating performance metrics...")
    
    # Fix datetime types
    forecasts['ds'] = pd.to_datetime(forecasts['ds'])
    test_data['ds'] = pd.to_datetime(test_data['ds'])
    
    # Merge forecasts with test data
    evaluation_data = pd.merge(
        test_data,
        forecasts[['ds', 'unique_id', 'TimeGPT']],
        on=['ds', 'unique_id'],
        how='inner'
    )
    
    if len(evaluation_data) == 0:
        print("   ❌ No matching data for evaluation")
        return None
    
    # Check for zero values in actual data
    zero_count = (evaluation_data['y'] == 0).sum()
    near_zero_count = (np.abs(evaluation_data['y']) < 1e-8).sum()
    
    print(f"   📊 Data quality check:")
    print(f"      • Zero values in actual data: {zero_count}")
    print(f"      • Near-zero values: {near_zero_count}")
    
    # Calculate metrics by product
    product_metrics = []
    
    for product_id in evaluation_data['unique_id'].unique():
        product_eval = evaluation_data[evaluation_data['unique_id'] == product_id]
        
        if len(product_eval) > 0:
            # Standard metrics
            mse = mean_squared_error(product_eval['y'], product_eval['TimeGPT'])
            mae = mean_absolute_error(product_eval['y'], product_eval['TimeGPT'])
            rmse = np.sqrt(mse)
            
            # Improved MAPE calculation with zero handling
            actual_values = product_eval['y'].values
            predicted_values = product_eval['TimeGPT'].values
            
            # Method 1: Exclude zero values from MAPE calculation
            non_zero_mask = actual_values != 0
            if non_zero_mask.sum() > 0:
                mape = np.mean(np.abs((actual_values[non_zero_mask] - predicted_values[non_zero_mask]) / actual_values[non_zero_mask])) * 100
                effective_points = non_zero_mask.sum()
            else:
                mape = np.inf  # Still infinity if all values are zero
                effective_points = 0
            
            # Alternative: sMAPE (Symmetric MAPE) - more robust to zeros
            smape = np.mean(2 * np.abs(actual_values - predicted_values) / 
                           (np.abs(actual_values) + np.abs(predicted_values))) * 100
            
            # Alternative: WAPE (Weighted Absolute Percentage Error)
            if actual_values.sum() != 0:
                wape = np.sum(np.abs(actual_values - predicted_values)) / np.sum(np.abs(actual_values)) * 100
            else:
                wape = np.inf
            
            product_metrics.append({
                'product': product_id,
                'mse': mse,
                'mae': mae,
                'rmse': rmse,
                'mape': mape,
                'smape': smape,  # Symmetric MAPE - better for zeros
                'wape': wape,    # Weighted APE - alternative metric
                'data_points': len(product_eval),
                'effective_points': effective_points,  # Points used in MAPE calculation
                'zero_values': (actual_values == 0).sum()
            })
    
    metrics_df = pd.DataFrame(product_metrics)
    
    # Calculate overall statistics using alternative metrics when MAPE is problematic
    finite_mape_mask = np.isfinite(metrics_df['mape'])
    
    if finite_mape_mask.sum() > 0:
        avg_mape = metrics_df.loc[finite_mape_mask, 'mape'].mean()
        mape_note = f"(calculated from {finite_mape_mask.sum()}/{len(metrics_df)} products)"
    else:
        avg_mape = np.inf
        mape_note = "(all products have zero values - use sMAPE instead)"
    
    overall_stats = {
        'avg_mse': metrics_df['mse'].mean(),
        'avg_mae': metrics_df['mae'].mean(),
        'avg_rmse': metrics_df['rmse'].mean(),
        'avg_mape': avg_mape,
        'avg_smape': metrics_df['smape'].mean(),  # More reliable alternative
        'avg_wape': metrics_df[np.isfinite(metrics_df['wape'])]['wape'].mean(),
        'total_products': len(metrics_df),
        'total_points': len(evaluation_data),
        'products_with_zeros': (metrics_df['zero_values'] > 0).sum()
    }
    
    print(f"   ✅ Evaluation complete: {len(metrics_df)} products evaluated")
    print(f"   📊 Average MAE: {overall_stats['avg_mae']:.2f}")
    print(f"   📊 Average RMSE: {overall_stats['avg_rmse']:.2f}")
    print(f"   📊 Average sMAPE: {overall_stats['avg_smape']:.2f}% (recommended)")
    print(f"   📊 Average MAPE: {overall_stats['avg_mape']:.2f}% {mape_note}")
    print(f"   ⚠️  Products with zero values: {overall_stats['products_with_zeros']}")
    
    return {
        'product_metrics': metrics_df,
        'overall_stats': overall_stats,
        'evaluation_data': evaluation_data
    }

def create_comprehensive_visualizations(full_data, forecasts, test_data, evaluation_results):
    """
    Create comprehensive visualizations using TimeGPT's built-in plotting
    """
    print("   • Creating TimeGPT visualization...")
    
    if evaluation_results is None:
        print("   ❌ No evaluation data for visualization")
        return
    
    # Use TimeGPT's built-in plotting function
    try:
        # Plot using TimeGPT's native plotting with zoomed view
        timegpt.plot(
            test_data,
            forecasts,
            models=["TimeGPT"],
            level=[90],
            time_col="ds",
            target_col="y",
            id_col="unique_id",
            max_insample_length=60  # Show last 60 days for context
        )
        
        print("   ✅ TimeGPT native plot created")
        
    except Exception as e:
        print(f"   ⚠️ TimeGPT plot failed: {e}")
        print("   🔄 Creating custom matplotlib visualization...")
        
        # Fallback to custom matplotlib plot
        create_custom_visualization(full_data, forecasts, test_data, evaluation_results)

def create_custom_visualization(full_data, forecasts, test_data, evaluation_results):
    """
    Create custom matplotlib visualization
    """
    evaluation_data = evaluation_results['evaluation_data']
    product_metrics = evaluation_results['product_metrics']
    
    # Get top 3 products for plotting
    top_products = product_metrics.nsmallest(3, 'mape')['product'].tolist()
    
    fig, axes = plt.subplots(len(top_products), 1, figsize=(15, 5*len(top_products)))
    if len(top_products) == 1:
        axes = [axes]
    
    for idx, product_id in enumerate(top_products):
        # Get data for this product
        product_train = full_data[full_data['unique_id'] == product_id]
        product_test = evaluation_data[evaluation_data['unique_id'] == product_id]
        product_pred = forecasts[forecasts['unique_id'] == product_id]
        
        # Get metrics
        metrics = product_metrics[product_metrics['product'] == product_id].iloc[0]
        
        # Find split point
        if len(product_test) > 0:
            test_start = product_test['ds'].min()
            train_plot = product_train[product_train['ds'] >= test_start - pd.Timedelta(days=30)]
        
        # Plot training data (last 30 days)
        axes[idx].plot(train_plot['ds'], train_plot['y'], 'b-', linewidth=2, label='Training Data', alpha=0.7)
        
        # Plot actual test data
        axes[idx].plot(product_test['ds'], product_test['y'], 'go-', linewidth=3, markersize=8, label='Actual Test Data')
        
        # Plot predictions
        axes[idx].plot(product_pred['ds'], product_pred['TimeGPT'], 'r--', linewidth=3, marker='s', markersize=8, label='TimeGPT Predictions')
        
        # Add confidence intervals if available
        if 'TimeGPT-lo-90' in product_pred.columns:
            axes[idx].fill_between(product_pred['ds'], product_pred['TimeGPT-lo-90'], product_pred['TimeGPT-hi-90'], color='red', alpha=0.2, label='90% Confidence')
        
        # Formatting
        title = f'Product: {product_id[:50]}...\n'
        title += f'MAPE: {metrics["mape"]:.1f}% | MAE: {metrics["mae"]:.1f}'
        
        axes[idx].set_title(title, fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('Date')
        axes[idx].set_ylabel('Sales')
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)
        
        # Format dates
        axes[idx].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
        plt.setp(axes[idx].xaxis.get_majorticklabels(), rotation=45)
    
    plt.suptitle('🤖 TimeGPT Forecasting Results', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("   ✅ Custom visualization created")

def generate_future_forecasts(data, horizon=14):
    """
    Generate future forecasts
    """
    print(f"   • Generating {horizon}-day future forecasts...")
    
    try:
        future_forecasts = timegpt.forecast(
            df=data,
            h=horizon,
            freq='D',
            level=[80, 90],
            time_col='ds',
            target_col='y',
            id_col='unique_id'
        )
        
        # Create summary
        forecast_summary = future_forecasts.groupby('unique_id').agg({
            'TimeGPT': ['mean', 'sum']
        }).round(2)
        
        forecast_summary.columns = ['avg_daily', 'total_forecast']
        forecast_summary = forecast_summary.sort_values('total_forecast', ascending=False)
        
        print(f"   ✅ Future forecasts generated")
        print(f"   📊 Top 5 products by projected sales:")
        print(forecast_summary.head())
        
        return future_forecasts
        
    except Exception as e:
        print(f"   ❌ Future forecast error: {e}")
        return None

# Main execution function

def run_timegpt_forecasting_fixed(train_data, predictable_features, test_periods=7):
    """
    FIXED TimeGPT forecasting with correct exogenous variable format
    """
    print(f"   🎯 Running FIXED TimeGPT forecast for {test_periods} periods...")
    
    try:
        # ✅ CORRECT APPROACH: Keep all variables in the same dataframe
        available_exog = [f for f in predictable_features if f in train_data.columns]
        
        if available_exog:
            print(f"   • Using exogenous variables: {available_exog}")
            
            # Use the complete dataframe with target + exogenous variables
            forecast_data = train_data[['unique_id', 'ds', 'y'] + available_exog].copy()
            
            print(f"   • Forecast data shape: {forecast_data.shape}")
            print(f"   • Columns: {list(forecast_data.columns)}")
            
            # Generate forecasts with all variables in one dataframe
            forecasts = timegpt.forecast(
                df=forecast_data,     # ✅ Single dataframe with target + exogenous
                h=test_periods,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id',
                # Fine-tuning parameters
                finetune_steps=30,
                finetune_loss='mae',
                clean_ex_first=True
            )
            
        else:
            print("   • No exogenous variables available, using target only")
            forecast_data = train_data[['unique_id', 'ds', 'y']].copy()
            
            forecasts = timegpt.forecast(
                df=forecast_data,
                h=test_periods,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id',
                finetune_steps=30,
                finetune_loss='mae'
            )
        
        print(f"   ✅ FIXED forecasts generated: {len(forecasts):,} rows")
        return forecasts
        
    except Exception as e:
        print(f"   ❌ Fixed forecasting failed: {e}")
        print("   🔄 Trying basic approach without exogenous variables...")
        
        # Fallback: Use only target variables
        try:
            basic_data = train_data[['unique_id', 'ds', 'y']].copy()
            
            forecasts = timegpt.forecast(
                df=basic_data,
                h=test_periods,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id',
                finetune_steps=20,
                finetune_loss='mae'
            )
            
            print(f"   ✅ Basic forecasts generated: {len(forecasts):,} rows")
            return forecasts
            
        except Exception as e2:
            print(f"   ❌ All approaches failed: {e2}")
            
            # Final fallback - minimal configuration
            minimal_data = train_data[['unique_id', 'ds', 'y']].head(1000)  # Reduce data size
            
            forecasts = timegpt.forecast(
                df=minimal_data,
                h=test_periods,
                freq='D',
                time_col='ds',
                target_col='y',
                id_col='unique_id'
            )
            
            print(f"   ✅ Minimal fallback forecasts: {len(forecasts):,} rows")
            return forecasts
def generate_future_forecasts_fixed(data, predictable_features, horizon=14):
    """
    Generate future forecasts with proper exogenous variable handling - CORRECTED VERSION
    """
    print(f"   • Generating {horizon}-day future forecasts...")
    
    try:
        # Check if exogenous variables exist
        available_exog = [col for col in predictable_features if col in data.columns]
        
        if available_exog:
            print(f"   • Including exogenous features: {available_exog}")
            
            # ✅ CORRECT APPROACH: Separate historical data and future exogenous data
            historical_data = data[['unique_id', 'ds', 'y'] + available_exog].copy()
            
            # Create future exogenous data WITHOUT target variable
            future_X_df = create_future_exogenous_data_fixed(data, available_exog, horizon)
            
            print(f"   • Historical data shape: {historical_data.shape}")
            print(f"   • Future exogenous data shape: {future_X_df.shape}")
            
            # Use TimeGPT with separate dataframes
            future_forecasts = timegpt.forecast(
                df=historical_data,    # ✅ Historical data with target
                X_df=future_X_df,      # ✅ Future exogenous data WITHOUT target
                h=horizon,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id'
            )
        else:
            # No exogenous variables - standard forecasting
            future_forecasts = timegpt.forecast(
                df=data[['unique_id', 'ds', 'y']],
                h=horizon,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id'
            )
        
        # Create summary
        forecast_summary = future_forecasts.groupby('unique_id').agg({
            'TimeGPT': ['mean', 'sum']
        }).round(2)
        
        forecast_summary.columns = ['avg_daily', 'total_forecast']
        forecast_summary = forecast_summary.sort_values('total_forecast', ascending=False)
        
        print(f"   ✅ Future forecasts generated")
        print(f"   📊 Top 5 products by projected sales:")
        print(forecast_summary.head())
        
        return future_forecasts
        
    except Exception as e:
        print(f"   ❌ Future forecast error: {e}")
        return None


def create_future_exogenous_data_fixed(train_data, exog_features, horizon):
    """
    Create future exogenous variable values for forecasting - FIXED VERSION
    """
    future_data = []
    
    for unique_id in train_data['unique_id'].unique():
        # Get last date for this product
        product_data = train_data[train_data['unique_id'] == unique_id]
        last_date = product_data['ds'].max()
        
        # Generate future dates
        future_dates = pd.date_range(
            start=last_date + pd.Timedelta(days=1),
            periods=horizon,
            freq='D'
        )
        
        for date in future_dates:
            row = {'unique_id': unique_id, 'ds': date}  # y is None for future periods
            
            # Add predictable calendar features
            if 'year' in exog_features:
                row['year'] = date.year
            if 'month' in exog_features:
                row['month'] = date.month
            if 'day_of_week' in exog_features:
                row['day_of_week'] = date.dayofweek
            if 'is_weekend' in exog_features:
                row['is_weekend'] = int(date.dayofweek >= 5)
            if 'is_month_start' in exog_features:
                row['is_month_start'] = int(date.is_month_start)
            if 'is_month_end' in exog_features:
                row['is_month_end'] = int(date.is_month_end)
            if 'is_quarter_start' in exog_features:
                row['is_quarter_start'] = int(date.is_quarter_start)
            if 'is_quarter_end' in exog_features:
                row['is_quarter_end'] = int(date.is_quarter_end)
            
            # Add holiday indicators (extend for future years)
            if 'is_republic_day' in exog_features:
                row['is_republic_day'] = int(date.strftime('%m-%d') == '01-26')
            if 'is_holi' in exog_features:
                row['is_holi'] = 0  # Add logic for variable date holidays
            if 'is_diwali' in exog_features:
                row['is_diwali'] = 0  # Add logic for variable date holidays
            
            future_data.append(row)
    
    return pd.DataFrame(future_data)


# Updated main execution function
def complete_timegpt_pipeline_with_exogenous_fixed(df_sorted, top_n_products=8):
    """
    Complete end-to-end TimeGPT forecasting pipeline with exogenous features - FIXED VERSION
    """
    print("🚀 STARTING FIXED TIMEGPT PIPELINE WITH EXOGENOUS FEATURES")
    print("=" * 70)
    
    # Step 1: Enhanced Data Cleaning and Preparation with Exogenous Features
    print("\n📋 STEP 1: DATA CLEANING WITH EXOGENOUS FEATURES")
    cleaned_data, predictable_features = clean_and_prepare_data_with_exogenous(df_sorted, top_n_products)
    
    # Step 2: Fix Frequency Issues
    print("\n🔧 STEP 2: FIXING FREQUENCY ISSUES")
    frequency_fixed_data = fix_frequency_issues_complete(cleaned_data)
    
    # Step 3: Train-Test Split
    print("\n✂️ STEP 3: TRAIN-TEST SPLIT")
    train_data, test_data = create_train_test_split(frequency_fixed_data)
    
    # Step 4: TimeGPT Forecasting with Exogenous Features - FIXED
    print("\n🤖 ST`EP 4: TIMEGPT FORECASTING WITH EXOGENOUS FEATURES - FIXED")
    forecasts = run_timegpt_forecasting_fixed(train_data, predictable_features, test_periods=7)
    
    # Step 5: Evaluation
    print("\n📊 STEP 5: MODEL EVALUATION")
    evaluation_results = evaluate_forecasts(forecasts, test_data)
    
    # Step 6: Visualization
    print("\n📈 STEP 6: CREATING VISUALIZATIONS")
    create_comprehensive_visualizations(frequency_fixed_data, forecasts, test_data, evaluation_results)
    
    # Step 7: Future Forecasting - FIXED
    print("\n🔮 STEP 7: FUTURE FORECASTING - FIXED")
    future_forecasts = generate_future_forecasts_fixed(frequency_fixed_data, predictable_features, horizon=14)
    
    return {
        'cleaned_data': frequency_fixed_data,
        'train_data': train_data,
        'test_data': test_data,
        'forecasts': forecasts,
        'evaluation': evaluation_results,
        'future_forecasts': future_forecasts,
        'exogenous_features': predictable_features
    }



results = complete_timegpt_pipeline_with_exogenous_fixed(df_sorted, top_n_products=8)


🚀 STARTING FIXED TIMEGPT PIPELINE WITH EXOGENOUS FEATURES

📋 STEP 1: DATA CLEANING WITH EXOGENOUS FEATURES
   • Creating exogenous features before data aggregation...
   • Created 11 predictable exogenous features
   • Found 12853 negative sales records
   📊 Product filtering with exogenous features:
      • Viable for forecasting: 381
      • Selected top: 8
   • Aggregating with enhanced exogenous variables...


INFO:nixtlats.nixtla_client:Validating inputs...
INFO:nixtlats.nixtla_client:Preprocessing dataframes...


   ✅ Enhanced data with exogenous features: 6,245 rows, 8 products
   📊 Exogenous features included: 11

🔧 STEP 2: FIXING FREQUENCY ISSUES
   • Applying robust frequency correction for irregular data...
      Processing: 04042 | MENS WEAR | SMART CASUALS | BOTT... (790 records)
        📊 Missing ratio: 0.25%
        ✅ Success: 2 gaps filled
      Processing: 04042 | MENS WEAR | SMART CASUALS | TOPS... (792 records)
        📊 Missing ratio: 0.00%
        ✅ Success: 0 gaps filled
      Processing: 04063 | MENS CASUAL | ACTIVE WEAR | BOTT... (791 records)
        📊 Missing ratio: 0.13%
        ✅ Success: 1 gaps filled
      Processing: 04063 | MENS CASUAL | ACTIVE WEAR | TOPS... (792 records)
        📊 Missing ratio: 0.00%
        ✅ Success: 0 gaps filled
      Processing: D699 | WOMENS WEAR | ETHNIC WEAR | TOPWE... (788 records)
        📊 Missing ratio: 0.51%
        ✅ Success: 4 gaps filled
      Processing: D699 | WOMENS WEAR | ETHNIC WEAR | TOPWE... (709 records)
        📊 Missing rat

INFO:nixtlats.nixtla_client:Calling Forecast Endpoint...
INFO:nixtlats.nixtla_client:Validating inputs...
INFO:nixtlats.nixtla_client:Preprocessing dataframes...


   ✅ FIXED forecasts generated: 56 rows

📊 STEP 5: MODEL EVALUATION
   • Calculating performance metrics...
   📊 Data quality check:
      • Zero values in actual data: 0
      • Near-zero values: 0
   ✅ Evaluation complete: 8 products evaluated
   📊 Average MAE: 6776.82
   📊 Average RMSE: 8344.29
   📊 Average sMAPE: 36.98% (recommended)
   📊 Average MAPE: 41.46% (calculated from 8/8 products)
   ⚠️  Products with zero values: 0

📈 STEP 6: CREATING VISUALIZATIONS
   • Creating TimeGPT visualization...
   ✅ TimeGPT native plot created

🔮 STEP 7: FUTURE FORECASTING - FIXED
   • Generating 14-day future forecasts...
   • Including exogenous features: ['year', 'month', 'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'is_republic_day', 'is_holi', 'is_diwali']
   • Historical data shape: (6335, 14)
   • Future exogenous data shape: (112, 13)
   ❌ Future forecast error: Some of your exogenous variables contain NA, please check


In [34]:
if results:
    cleaned_data = results['cleaned_data']
    forecasts = results['forecasts']
    evaluation = results['evaluation']
    future_forecasts = results['future_forecasts']
    
    # Print performance summary
    if evaluation:
        print(f"Model Performance: {evaluation['overall_stats']['avg_mape']:.1f}% MAPE")

Model Performance: 87.1% MAPE


In [12]:
import pandas as pd

# 1. Ensure BILLING_DATE is datetime
df = df_sorted.copy()
df['BILLING_DATE'] = pd.to_datetime(df['BILLING_DATE'])

# 2. Date range
min_date = df['BILLING_DATE'].min()
max_date = df['BILLING_DATE'].max()
print(f"Date range: {min_date.date()} → {max_date.date()}")

# 3. Granularity (frequency)  
#    We infer on the full date index; if irregular, you may want to sample a single product.
freq = pd.infer_freq(df.set_index('BILLING_DATE').sort_index().index)
print(f"Overall inferred frequency: {freq}")

# 4. Unique series count & typical length
num_products = df['PRODUCT_KEY'].nunique()
lengths = df.groupby('PRODUCT_KEY')['BILLING_DATE'].nunique()
print(f"Unique products: {num_products}")
print(f"History length per product (days):")
print(f"  • min: {lengths.min()},  median: {lengths.median()},  max: {lengths.max()}")

# 5. Target distribution
print("\nTOTAL_NET_SALES distribution:")
print(df['TOTAL_NET_SALES'].describe())

# 6. Missing / zero values
print("\nMissing values per column:")
print(df.isna().sum())
print(f"\nDays with zero sales: {(df['TOTAL_NET_SALES'] == 0).sum()}")

# 7. List of potential covariates
print("\nAvailable columns for exogenous features:")
print([c for c in df.columns if c not in ['BILLING_DATE', 'TOTAL_NET_SALES', 'PRODUCT_KEY']])


Date range: 2022-09-30 → 2024-11-29
Overall inferred frequency: None
Unique products: 1883
History length per product (days):
  • min: 1,  median: 18.0,  max: 792

TOTAL_NET_SALES distribution:
count    823225.000000
mean        723.379285
std         798.541777
min       -6999.000000
25%         399.000000
50%         599.000000
75%         915.570000
max      214500.000000
Name: TOTAL_NET_SALES, dtype: float64

Missing values per column:
Unnamed: 0            0
BILLING_DATE          0
SITE                  0
BRAND                 0
FORMAT_DESC           0
MH_SEGMENT            0
MH_FAMILY             0
MH_CLASS              0
MH_BRICK              0
ARTICLE               0
TOTAL_BILLING_QTY     0
TOTAL_GROSS_MARGIN    0
TOTAL_MARKDOWN        0
TOTAL_NET_SALES       0
TOTAL_GROSS_SALES     0
TOTAL_DISCOUNT        0
PRODUCT_KEY           0
dtype: int64

Days with zero sales: 1233

Available columns for exogenous features:
['Unnamed: 0', 'SITE', 'BRAND', 'FORMAT_DESC', 'MH_SEGMENT', 'MH

In [13]:

MIN_HISTORY = 60
hist_lengths = df.groupby('PRODUCT_KEY')['BILLING_DATE'].nunique()
valid_keys = hist_lengths[hist_lengths >= MIN_HISTORY].index
df = df[df['PRODUCT_KEY'].isin(valid_keys)]
print(f"Kept {df['PRODUCT_KEY'].nunique()} products with ≥{MIN_HISTORY} days of data")


Kept 552 products with ≥60 days of data


In [46]:
def create_calendar_features(df):
    """Create comprehensive calendar features for Indian retail"""
    df = df.copy()
    df['BILLING_DATE'] = pd.to_datetime(df['BILLING_DATE'])
    
    # Basic temporal features
    df['year'] = df['BILLING_DATE'].dt.year
    df['month'] = df['BILLING_DATE'].dt.month
    df['day_of_week'] = df['BILLING_DATE'].dt.dayofweek
    df['day_of_year'] = df['BILLING_DATE'].dt.dayofyear
    df['week_of_year'] = df['BILLING_DATE'].dt.isocalendar().week
    df['quarter'] = df['BILLING_DATE'].dt.quarter
    
    # Weekend indicator
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    
    # Month-end/start effects
    df['is_month_start'] = df['BILLING_DATE'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['BILLING_DATE'].dt.is_month_end.astype(int)
    df['is_quarter_start'] = df['BILLING_DATE'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['BILLING_DATE'].dt.is_quarter_end.astype(int)
    
    return df


In [64]:
timegpt_data = (
    df.rename(columns={'BILLING_DATE':'ds', 'TOTAL_NET_SALES':'y'})
      .set_index('ds')
      .groupby('PRODUCT_KEY')['y']
      .resample('W-MON')   # weekly bins starting Monday
      .sum()
      .reset_index()
      .rename(columns={'PRODUCT_KEY':'unique_id'})
)
print(f"Weekly series: {timegpt_data['unique_id'].nunique()} products, {len(timegpt_data)} rows")


KeyError: 'PRODUCT_KEY'

In [29]:
import pandas as pd

# 1. Ensure BILLING_DATE is datetime
df = df_sorted.copy()
df['BILLING_DATE'] = pd.to_datetime(df['BILLING_DATE'])

# 2. Filter out short series (< MIN_HISTORY days)
MIN_HISTORY = 60
hist_lengths = df.groupby('PRODUCT_KEY')['BILLING_DATE'].nunique()
valid_keys = hist_lengths[hist_lengths >= MIN_HISTORY].index
df = df[df['PRODUCT_KEY'].isin(valid_keys)]
print(f"Kept {df['PRODUCT_KEY'].nunique()} products with ≥{MIN_HISTORY} days of data")

# 3. Create the TimeGPT “y” series and aggregate weekly
#    (sum of TOTAL_NET_SALES per week starting Monday)
timegpt_data = (
    df.rename(columns={'BILLING_DATE':'ds', 'TOTAL_NET_SALES':'y'})
      .set_index('ds')
      .groupby('PRODUCT_KEY')['y']
      .resample('W-MON')   # weekly bins starting Monday
      .sum()
      .reset_index()
      .rename(columns={'PRODUCT_KEY':'unique_id'})
)
print(f"Weekly series: {timegpt_data['unique_id'].nunique()} products, {len(timegpt_data)} rows")

# 4. Build exogenous DataFrame X_df, aggregated the same way
#    Here we include qty, discount, and holiday flag (example)
#    First add weekday/month/holiday columns on the original df
df['dayofweek'] = df['BILLING_DATE'].dt.dayofweek
df['is_holiday'] = df['BILLING_DATE'].isin(pd.to_datetime([
    '2023-01-26','2023-08-15','2024-01-26',  # add your full holiday list
])).astype(int)

#   Now aggregate covariates weekly
covariates = ['TOTAL_BILLING_QTY','TOTAL_DISCOUNT','dayofweek','is_holiday']
X_df = (
    df.set_index('BILLING_DATE')
      .groupby('PRODUCT_KEY')[covariates]
      .resample('W-MON')
      .agg({
         'TOTAL_BILLING_QTY':'sum',
         'TOTAL_DISCOUNT':'sum',
         'dayofweek':'mean',      # average weekday in the week
         'is_holiday':'sum'       # count of holidays that week
      })
      .reset_index()
      .rename(columns={'PRODUCT_KEY':'unique_id', 'BILLING_DATE':'ds'})
)
print(f"Exogenous features X_df: {X_df.shape[0]} rows, columns: {covariates}")

# 5. Inspect final shapes before forecasting
print("\nSample of y-series:")
print(timegpt_data.head())
print("\nSample of X_df:")
print(X_df.head())


Kept 552 products with ≥60 days of data
Weekly series: 552 products, 42417 rows
Exogenous features X_df: 42417 rows, columns: ['TOTAL_BILLING_QTY', 'TOTAL_DISCOUNT', 'dayofweek', 'is_holiday']

Sample of y-series:
                                           unique_id         ds       y
0  02156 | PERSONAL CARE | P'SONAL HYGN | GENRL P... 2023-07-17   750.0
1  02156 | PERSONAL CARE | P'SONAL HYGN | GENRL P... 2023-07-24  1125.0
2  02156 | PERSONAL CARE | P'SONAL HYGN | GENRL P... 2023-07-31  1125.0
3  02156 | PERSONAL CARE | P'SONAL HYGN | GENRL P... 2023-08-07   375.0
4  02156 | PERSONAL CARE | P'SONAL HYGN | GENRL P... 2023-08-14     0.0

Sample of X_df:
                                           unique_id         ds  \
                                                                  
0  02156 | PERSONAL CARE | P'SONAL HYGN | GENRL P... 2023-07-17   
1  02156 | PERSONAL CARE | P'SONAL HYGN | GENRL P... 2023-07-24   
2  02156 | PERSONAL CARE | P'SONAL HYGN | GENRL P... 2023-07-31   
3 

In [62]:
#EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Load your data
df = pd.read_csv("data-8013-trends.csv")

print("="*60)
print("DATA EXPLORATION FOR TIMEGPT PREPARATION")
print("="*60)

# Basic data info
print("\n1. BASIC DATA STRUCTURE")
print("-" * 40)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"Data types:\n{df.dtypes}")

# Convert BILLING_DATE to datetime
df['BILLING_DATE'] = pd.to_datetime(df['BILLING_DATE'])

# Fix BRAND column - convert all to string to handle mixed types
df['BRAND'] = df['BRAND'].astype(str)

# Date range analysis
print("\n2. TIME SERIES ANALYSIS")
print("-" * 40)
print(f"Date range: {df['BILLING_DATE'].min()} to {df['BILLING_DATE'].max()}")
print(f"Total days span: {(df['BILLING_DATE'].max() - df['BILLING_DATE'].min()).days} days")
print(f"Unique dates: {df['BILLING_DATE'].nunique()}")

# Check for date gaps
date_range = pd.date_range(start=df['BILLING_DATE'].min(), 
                          end=df['BILLING_DATE'].max(), 
                          freq='D')
missing_dates = set(date_range) - set(df['BILLING_DATE'].unique())
print(f"Missing dates: {len(missing_dates)}")
if len(missing_dates) > 0 and len(missing_dates) < 10:
    print(f"Missing dates: {sorted(missing_dates)[:10]}")

# Daily transaction counts
daily_transactions = df.groupby('BILLING_DATE').size()
print(f"Average transactions per day: {daily_transactions.mean():.1f}")
print(f"Min transactions per day: {daily_transactions.min()}")
print(f"Max transactions per day: {daily_transactions.max()}")

print("\n3. BUSINESS DIMENSIONS")
print("-" * 40)
print(f"Unique SITES: {df['SITE'].nunique()}")
print(f"Unique BRANDS: {df['BRAND'].nunique()}")
print(f"Unique ARTICLES: {df['ARTICLE'].nunique()}")
print(f"Sites: {sorted(df['SITE'].unique())}")
print(f"Brands: {sorted(df['BRAND'].unique())[:10]}")  # Show first 10 brands only

print("\n4. AGGREGATION POSSIBILITIES")
print("-" * 40)

# Daily aggregations by different levels
print("A. DAILY TOTALS - OVERALL:")
daily_overall = df.groupby('BILLING_DATE').agg({
    'TOTAL_NET_SALES': 'sum',
    'TOTAL_GROSS_SALES': 'sum', 
    'TOTAL_BILLING_QTY': 'sum'
}).round(2)

print(daily_overall.head())
print(f"Daily NET SALES range: {daily_overall['TOTAL_NET_SALES'].min():.2f} to {daily_overall['TOTAL_NET_SALES'].max():.2f}")

print("\nB. DAILY TOTALS - BY SITE:")
daily_by_site = df.groupby(['BILLING_DATE', 'SITE']).agg({
    'TOTAL_NET_SALES': 'sum',
    'TOTAL_BILLING_QTY': 'sum'
}).reset_index()

for site in df['SITE'].unique()[:3]:  # Show first 3 sites
    site_data = daily_by_site[daily_by_site['SITE'] == site]
    print(f"Site {site}: {len(site_data)} days, Avg daily sales: {site_data['TOTAL_NET_SALES'].mean():.2f}")

print("\nC. DAILY TOTALS - BY BRAND:")
daily_by_brand = df.groupby(['BILLING_DATE', 'BRAND']).agg({
    'TOTAL_NET_SALES': 'sum',
    'TOTAL_BILLING_QTY': 'sum'
}).reset_index()

# Get top 5 brands by total sales to avoid issues with 'nan' brands
top_brands = df.groupby('BRAND')['TOTAL_NET_SALES'].sum().sort_values(ascending=False).head(5).index

for brand in top_brands:
    if brand != 'nan':  # Skip if brand is 'nan' string
        brand_data = daily_by_brand[daily_by_brand['BRAND'] == brand]
        if len(brand_data) > 0:
            print(f"Brand {brand}: {len(brand_data)} days, Avg daily sales: {brand_data['TOTAL_NET_SALES'].mean():.2f}")

print("\n5. DATA QUALITY CHECK")
print("-" * 40)
print("Missing values:")
print(df.isnull().sum())

print("\nBrand data quality:")
print(f"Brands with 'nan' string: {(df['BRAND'] == 'nan').sum()}")
print(f"Most common brands:")
print(df['BRAND'].value_counts().head())

print("\nZero/Negative values in key metrics:")
for col in ['TOTAL_NET_SALES', 'TOTAL_GROSS_SALES', 'TOTAL_BILLING_QTY']:
    zero_count = (df[col] == 0).sum()
    neg_count = (df[col] < 0).sum()
    print(f"{col}: {zero_count} zeros, {neg_count} negative values")

print("\n6. POTENTIAL FORECASTING TARGETS")
print("-" * 40)

# Create different aggregation examples
forecasting_options = {
    'daily_total_sales': df.groupby('BILLING_DATE')['TOTAL_NET_SALES'].sum(),
    'daily_total_quantity': df.groupby('BILLING_DATE')['TOTAL_BILLING_QTY'].sum(),
    'daily_transaction_count': df.groupby('BILLING_DATE').size()
}

for name, series in forecasting_options.items():
    print(f"\n{name.upper()}:")
    print(f"  Date range: {series.index.min()} to {series.index.max()}")
    print(f"  Mean: {series.mean():.2f}")
    print(f"  Std: {series.std():.2f}")
    print(f"  Sample values: {series.head(3).to_dict()}")

print("\n7. WEEKLY PATTERNS")
print("-" * 40)
df['day_of_week'] = df['BILLING_DATE'].dt.day_name()
weekly_sales = df.groupby('day_of_week')['TOTAL_NET_SALES'].sum().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])
print("Sales by day of week:")
print(weekly_sales)

print("\n8. TIMEGPT DATA FORMAT EXAMPLES")
print("-" * 40)
print("TimeGPT requires format: ['unique_id', 'ds', 'y']")
print("\nOption 1 - Overall daily sales:")
timegpt_format1 = df.groupby('BILLING_DATE')['TOTAL_NET_SALES'].sum().reset_index()
timegpt_format1.columns = ['ds', 'y']
timegpt_format1['unique_id'] = 'total_sales'
timegpt_format1 = timegpt_format1[['unique_id', 'ds', 'y']]
print(timegpt_format1.head())

print("\nOption 2 - Daily sales by site:")
timegpt_format2 = df.groupby(['SITE', 'BILLING_DATE'])['TOTAL_NET_SALES'].sum().reset_index()
timegpt_format2.columns = ['unique_id', 'ds', 'y']
print(timegpt_format2.head())

print("\n" + "="*60)
print("NEXT STEPS: Answer these questions based on the output above:")
print("="*60)
print("1. Which forecasting target interests you most?")
print("   - daily_total_sales, daily_total_quantity, or daily_transaction_count?")
print("2. Do you want one overall forecast or separate forecasts by site/brand?")
print("3. How many days ahead do you want to forecast?")
print("4. Any data quality issues you want to address first?")
print("5. Do you see any concerning patterns in the date coverage?")


DATA EXPLORATION FOR TIMEGPT PREPARATION

1. BASIC DATA STRUCTURE
----------------------------------------
Dataset shape: (823225, 20)
Columns: ['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'BILLING_DATE', 'SITE', 'BRAND', 'FORMAT_DESC', 'MH_SEGMENT', 'MH_FAMILY', 'MH_CLASS', 'MH_BRICK', 'ARTICLE', 'TOTAL_BILLING_QTY', 'TOTAL_GROSS_MARGIN', 'TOTAL_MARKDOWN', 'TOTAL_NET_SALES', 'TOTAL_GROSS_SALES', 'TOTAL_DISCOUNT']
Data types:
Unnamed: 0.4            int64
Unnamed: 0.3            int64
Unnamed: 0.2            int64
Unnamed: 0.1            int64
Unnamed: 0              int64
BILLING_DATE           object
SITE                    int64
BRAND                  object
FORMAT_DESC            object
MH_SEGMENT             object
MH_FAMILY              object
MH_CLASS               object
MH_BRICK               object
ARTICLE                 int64
TOTAL_BILLING_QTY       int64
TOTAL_GROSS_MARGIN    float64
TOTAL_MARKDOWN        float64
TOTAL_NET_SALES       floa